In [1]:
import pandas as pd
import os
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
FILENAME = './data/audio_data/pifsc_audio_pipan_hawaii_pipan_hawaii_01_audio_Hawaii_K_01_070915_204730 (1).d20.x.wav'

model = hub.load('https://www.kaggle.com/models/google/humpback-whale/frameworks/TensorFlow2/variations/humpback-whale/versions/1')

waveform, sample_rate = tf.audio.decode_wav(tf.io.read_file(FILENAME))
waveform = tf.expand_dims(waveform, 0)  # makes a batch of size 1

pcen_spectrogram = model.front_end(waveform)
context_window = pcen_spectrogram[:, :128, :]
features = model.features(context_window)
# logits = model.logits(context_window)
# probabilities = tf.nn.sigmoid(logits)

# print({
#     'pcen_spectrogram': pcen_spectrogram,
#     'features': features,
#     'logits': logits,
#     'probabilities': probabilities,
# })

In [3]:
pcen_spectrogram

<tf.Tensor: shape=(1, 74997, 64), dtype=float32, numpy=
array([[[0.33115137, 0.30590713, 0.3021909 , ..., 0.28927267,
         0.26972795, 0.2688241 ],
        [0.24994373, 0.00690269, 0.0074352 , ..., 0.21948409,
         0.270568  , 0.3199687 ],
        [0.26100636, 0.00610518, 0.00963819, ..., 0.30254877,
         0.32813764, 0.22396529],
        ...,
        [0.32367277, 0.17629945, 0.16309893, ..., 0.2131772 ,
         0.21445918, 0.2696823 ],
        [0.33757567, 0.4267683 , 0.31122446, ..., 0.17577064,
         0.39610577, 0.3296262 ],
        [0.34199917, 0.41046095, 0.2801416 , ..., 0.3022095 ,
         0.2409172 , 0.2089529 ]]], dtype=float32)>

In [4]:
image = context_window.numpy()[0]
image.shape

(128, 64)

In [5]:
image.shape

(128, 64)

In [ ]:
plt.imshow(image)
print("here")
plt.show()

here


In [6]:
context_window

<tf.Tensor: shape=(1, 128, 64), dtype=float32, numpy=
array([[[0.33115137, 0.30590713, 0.3021909 , ..., 0.28927267,
         0.26972795, 0.2688241 ],
        [0.24994373, 0.00690269, 0.0074352 , ..., 0.21948409,
         0.270568  , 0.3199687 ],
        [0.26100636, 0.00610518, 0.00963819, ..., 0.30254877,
         0.32813764, 0.22396529],
        ...,
        [0.31022763, 0.02018857, 0.05152535, ..., 0.26447642,
         0.1469419 , 0.22390068],
        [0.3170154 , 0.03208816, 0.04877114, ..., 0.263677  ,
         0.39933264, 0.36529028],
        [0.31376207, 0.04851413, 0.02885354, ..., 0.22925687,
         0.4176612 , 0.34641027]]], dtype=float32)>

In [9]:
context_step_samples = tf.cast(sample_rate, tf.int64)
score_fn = model.signatures['score']
scores = score_fn(waveform=waveform, context_step_samples=context_step_samples)
print(scores)

{'scores': <tf.Tensor: shape=(1, 2269, 1), dtype=float32, numpy=
array([[[0.00462567],
        [0.00514184],
        [0.01479769],
        ...,
        [0.00103039],
        [0.00918357],
        [0.00408581]]], dtype=float32)>}


In [14]:
scores['scores'].numpy().flatten().max()

0.6983186

In [6]:
probabilities

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00462569]], dtype=float32)>

In [4]:
df = pd.read_csv("data/pifsc_products_detections_annotations.csv")
df[df['label'] != "Other"]

,audit_name,flac_compressed_xwav_object,subchunk_index,label_is_strong,implicit_negatives,label,begin_rel_subchunk,end_rel_subchunk,begin_utc,end_utc
18,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,11,True,True,Mn,48.174,49.138,2007-09-15 21:02:03.174000+00:00,2007-09-15 21:02:04.138000+00:00
19,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,11,True,True,Mn,52.431,53.132,2007-09-15 21:02:07.431000+00:00,2007-09-15 21:02:08.132000+00:00
20,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,12,True,True,Mn,46.230,46.738,2007-09-15 21:03:16.230000+00:00,2007-09-15 21:03:16.738000+00:00
21,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,12,True,True,Mn,49.926,50.679,2007-09-15 21:03:19.926000+00:00,2007-09-15 21:03:20.679000+00:00
22,initial,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,12,True,True,Mn,53.307,53.955,2007-09-15 21:03:23.307000+00:00,2007-09-15 21:03:23.955000+00:00
...,...,...,...,...,...,...,...,...,...,...
38852,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,910,False,True,Background,0.000,75.000,2016-04-27 10:00:00+00:00,2016-04-27 10:01:15+00:00
38853,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,1018,False,True,Background,0.000,75.000,2016-05-03 13:31:15+00:00,2016-05-03 13:32:30+00:00
38854,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,137,False,True,Background,0.000,75.000,2016-05-04 13:31:15+00:00,2016-05-04 13:32:30+00:00
38855,validation,gs://noaa-passive-bioacoustic/pifsc/audio/pipa...,430,False,True,Background,0.000,75.000,2016-05-13 01:30:00+00:00,2016-05-13 01:31:15+00:00


In [4]:
import tensorflow_hub

ModuleNotFoundError: No module named 'tensorflow_hub'

In [14]:
df['label'].value_counts()

label
Mn            32189
Background     5928
Other           306
Vessel          242
Fish             98
Device           94
Name: count, dtype: int64

In [ ]:
df

In [13]:
sorted(df['flac_compressed_xwav_object'].unique())

['gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050509_043845.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050510_210345.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050513_224615.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050514_014345.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050514_205230.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050516_013615.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050518_040230.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipan/crosssm/pipan_crosssm_01/audio/Cross_A_01_050519_000115.d20.x.flac',
 'gs://noaa-passive-bioacoustic/pifsc/audio/pipa